In [3]:
import datetime as dt
import awkward as ak
import pandas as pd
import pickle
import glob
import re
import h5py
import os

from legend200_data_loader.loader import LegendDataLoader
from legend200_data_loader.preprocessing import plot_detector_positions
from legend200_data_loader.function import filter_metadata
from legendmeta import LegendMetadata
from ipywidgets import interact, widgets
from IPython.display import display

#Set up SSH agent for authentication
os.environ["SSH_AUTH_SOCK"] = "/tmp/ssh-XXXXXX1U2ma8/agent.2354076"

#Load metadata
lmeta = LegendMetadata()

#Step 1: Metadata Filtering
detector_type = 'geds'  # Example: detector type to filter
detectors = filter_metadata(lmeta)
if not detectors:
    print(f"No detectors found for type: {detector_type}")
else:
    detector_entries = {key: value for key, value in detectors.items() if key != 'rawid_lists'}
    print(f"Found {len(detector_entries)} HPGE detectors.")

#Load data loader
data_loader = LegendDataLoader()

#Widget for selecting period
available_periods = ['p08/', 'p09/', 'p11/']  # Example list; replace with actual periods
period_dropdown = widgets.Dropdown(
    options=available_periods,
    value=available_periods[0],
    description='Period:'
)

#Widget for selecting run
run_dropdown = widgets.Dropdown(
    options=[],  # Updated dynamically based on period
    description='Run:'
)

#Filter options for detector usability
show_active = widgets.Checkbox(value=True, description='Show Active Detectors (on)')
show_ac = widgets.Checkbox(value=True, description='Show AC Detectors (ac)')
show_off = widgets.Checkbox(value=False, description='Show Off Detectors (off)')

#Display widgets
display(period_dropdown, run_dropdown, show_active, show_ac, show_off)

#Update runs based on the selected period
def update_runs(period):
    runs = data_loader.get_runs(period)
    run_dropdown.options = runs
    if runs:
        run_dropdown.value = runs[0]

#Update runs dropdown when period changes
period_dropdown.observe(lambda change: update_runs(change.new), names='value')
update_runs(period_dropdown.value)

save_plot_checkbox = widgets.Checkbox(value=False, description='Save Plot')
save_path_text = widgets.Text(
    value='/global/u2/m/mababicz/l200-detector-mapper/detector_plot.png',
    placeholder='Enter save path',
    description='Save Path:'
)

# Display the widgets for saving options
display(save_plot_checkbox, save_path_text)

#Function to update and plot detector positions
def update_plot(period, run, active, ac, off, save_plot, save_path):
    # Load files for selected period and run
    file_dict = data_loader.load_files(period, run)
    tier_to_check = 'raw_files'
    
    #Extract timestamps
    timestamps = []
    for file_path in file_dict.get(tier_to_check, []):
        filename = os.path.basename(file_path)
        match = re.search(r'\d{8}T\d{6}Z', filename)
        if match:
            timestamps.append(match.group())
    
    if not timestamps:
        print(f"No valid timestamps found in {tier_to_check} files.")
        return
    
    timestamp = timestamps[0]
    channel_map = lmeta.channelmap(timestamp)
    usable_detectors, ac_detectors, off_detectors = [], [], []
    
    for det_id in detectors:
        detector_info = channel_map.get(det_id, {})
        analysis_info = detector_info.get('analysis', {})
        usability = analysis_info.get('usability', 'unknown')
        
        if usability == 'on' and active:
            usable_detectors.append(det_id)
        elif usability == 'ac' and ac:
            ac_detectors.append(det_id)
        elif usability == 'off' and off:
            off_detectors.append(det_id)
    
    #Plot filtered detectors
    save_path_actual = save_path if save_plot else None
    plot_detector_positions(
        usable_detectors, 
        lmeta, 
        period, 
        run, 
        ac_detectors=ac_detectors, 
        off_detectors=off_detectors,
        #save_path=save_path_actual
    )
    if save_plot:
        print(f"Plot saved to {save_path}")

#Link widgets to the plot function
interact(
    update_plot,
    period=period_dropdown,
    run=run_dropdown,
    active=show_active,
    ac=show_ac,
    off=show_off,
    save_plot=save_plot_checkbox,
    save_path=save_path_text
)


Found 101 HPGE detectors.


Dropdown(description='Period:', options=('p08/', 'p09/', 'p11/'), value='p08/')

Dropdown(description='Run:', options=(), value=None)

Checkbox(value=True, description='Show Active Detectors (on)')

Checkbox(value=True, description='Show AC Detectors (ac)')

Checkbox(value=False, description='Show Off Detectors (off)')

Checkbox(value=False, description='Save Plot')

Text(value='/global/u2/m/mababicz/l200-detector-mapper/detector_plot.png', description='Save Path:', placehold…

interactive(children=(Dropdown(description='Period:', options=('p08/', 'p09/', 'p11/'), value='p08/'), Dropdow…

<function __main__.update_plot(period, run, active, ac, off, save_plot, save_path)>

In [4]:
print(plot_detector_positions.__module__)


legend200_data_loader.preprocessing
